In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import bq_helper # import bq_helper package

In [ ]:
# create a helper object for our bigquery dataset
bitcoin_blockchain = bq_helper.BigQueryHelper(active_project= "bigquery-public-data", 
                                       dataset_name = "bitcoin_blockchain")

# print a list of all the tables in the bitcoin_blockchain dataset
bitcoin_blockchain.list_tables()

In [ ]:






# print information on all the columns in the "full" table
# in the bitcoin_blockchain dataset
bitcoin_blockchain.table_schema("transactions")

In [ ]:
# preview the first couple lines of the "transactions" table
bitcoin_blockchain.head("transactions")

In [ ]:
# this query looks in the transactions table in the bitcoin_blockchain
# dataset, then gets the version column from every row where 
# the work_error column has "job" in it.
query = """SELECT t.block_id, COUNT(distinct t.transaction_id) C
            FROM `bigquery-public-data.bitcoin_blockchain.transactions` t
            GROUP BY t.block_id
            order by C desc
            """

# check how big this query will be; size in GB
bitcoin_blockchain.estimate_query_size(query)

In [ ]:
# only run this query if it's less than 100 MB
blockchain_txncount = bitcoin_blockchain.query_to_pandas_safe(query, max_gb_scanned=100)

In [ ]:
# this query looks in the transactions table in the bitcoin_blockchain
# dataset, then gets the version column from every row where 
# the work_error column has "job" in it.
query = """SELECT t.block_id, t.timestamp
            FROM `bigquery-public-data.bitcoin_blockchain.blocks` t
            """

# check how big this query will be; size in GB
bitcoin_blockchain.estimate_query_size(query)

In [ ]:
blockchain_timestamp = bitcoin_blockchain.query_to_pandas_safe(query, max_gb_scanned=100)

In [ ]:
blockchain_txncount_ts = pd.merge(blockchain_txncount, blockchain_timestamp, on='block_id', how='inner') 

In [ ]:
blockchain_txncount_ts.head()

In [ ]:
from subprocess import check_output
print(check_output(["ls", "../input/"]).decode("utf8"))



In [ ]:

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly as py
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
bitUSD = pd.read_csv('../input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2018-01-08.csv')

df_1 = blockchain_txncount_ts[['timestamp', 'C']].sort_values('timestamp', ascending=True)
df_1['timestamp'] = pd.to_datetime(df_1.timestamp/1000,unit='s')
df_1.head()

    

data = [go.Scatter(
            x=df_1['timestamp'],
            y=df_1['C'])]

layout = dict(
    title = "Time series plot of # Txns per block")
fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig)


In [ ]:
query = """
SELECT
  o.day,
  ROUND((SUM(o.output_price) / 100000000), 3) AS tot_BTC
FROM (
  SELECT
    TIMESTAMP_MILLIS((timestamp - MOD(timestamp,
          86400000))) AS day,
          block_id, transaction_id,
    output.output_satoshis AS output_price
  FROM
    `bigquery-public-data.bitcoin_blockchain.transactions`,
    UNNEST(outputs) AS output ) AS o
GROUP BY
  o.day
"""

# check how big this query will be; size in GB
bitcoin_blockchain.estimate_query_size(query)

In [ ]:
blockchain_tot_BTC_perday = bitcoin_blockchain.query_to_pandas_safe(query, max_gb_scanned=100)
blockchain_tot_BTC_perday.head()

In [ ]:
blockchain_tot_BTC_perday=blockchain_tot_BTC_perday.sort_values(by=["day"])
data = [go.Scatter(
            x=blockchain_tot_BTC_perday["day"],
            y=blockchain_tot_BTC_perday["tot_BTC"])]

layout = dict(
    title = "Daily bitcoins transacted")
fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig)

In [ ]:
query = """
SELECT
  o.day,
  count(Distinct transaction_id) txns
FROM (
  SELECT
    TIMESTAMP_MILLIS((timestamp - MOD(timestamp,
          86400000))) AS day,
         transaction_id
  FROM
    `bigquery-public-data.bitcoin_blockchain.transactions`,
    UNNEST(outputs) AS output ) AS o
GROUP BY
  o.day
"""

# check how big this query will be; size in GB
bitcoin_blockchain.estimate_query_size(query)

In [ ]:
blockchain_txns_perday = bitcoin_blockchain.query_to_pandas_safe(query, max_gb_scanned=100)
blockchain_txns_perday.head()


In [ ]:
blockchain_txns_perday=blockchain_txns_perday.sort_values(by=["day"])
data = [go.Scatter(
            x=blockchain_txns_perday["day"],
            y=blockchain_txns_perday["txns"])]

layout = dict(
    title = "Daily Count of bitcoin transactions")
fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig)

In [ ]:
output_addr_perday = pd.read_csv('../input/tempfile/tempfile2.csv')
output_addr_perday.head()

In [ ]:
blockchain_tot_BTC_perday=blockchain_tot_BTC_perday.sort_values(by=["day"])
data = [go.Scatter(
            x=output_addr_perday['day'],
            y=output_addr_perday["count"])]

layout = dict(
    title = "# Addresses receiving bitcoins per day")
fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig)